     |████████████████████████████████| 92kB 2.6MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.2 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [0]:
# install the latest version of requests
!pip install requests==v2.21.0
import requests
import pandas as pd
requests.__version__
# Install the latest release
!pip install folium==0.8.2
# check the version installed
import folium
folium.__version__
# Import the JSON library.
import json

# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

# from Sidra's Table 185 (http://api.sidra.ibge.gov.br/desctabapi.aspx?c=185)
endnode = "http://api.sidra.ibge.gov.br/values/t/185/p/2010/v/allxp/N102/in%20n6%202408102"

response = requests.get(endnode,headers=headers)

# for curiosity, take a look in this variable
raw_data = response.json()

neigh_id = []
neigh_name = []
neigh_police = []

# first position is only the header
for data in raw_data[1:]:
  neigh_id.append(int(data["D3C"]))
  neigh_name.append(data["D3N"].split(" -")[0])
  
# filling the number of units per neighborhood
for name in neigh_name:
  if(name == "Cidade da Esperança"):
    neigh_police.append(5)
  elif(name == "Lagoa Nova"):
    neigh_police.append(4)
  elif(name == "Potengi"):
    neigh_police.append(7)
  elif(name == "Tirol" or name == "Ribeira" or name == "Igapó"):
    neigh_police.append(2)
  elif(name == "Pitimbú" or name == "Filipe Camarão" or name == "Cidade Alta" or name ==	"Pajuçara"
          or name == "Mãe Luíza"  or name == "N Sra de Nazará" or name == "Barro Vermelho"
          or name == "Candelária" or name == "Rocas" or name == "Alecrim" or name == "Quintas"
          or name == 	"Neópolis" or name == "Bom Pastor" or name == "Capim Macio" or name == "Ponta Negra"):
    neigh_police.append(1)
  else:
    neigh_police.append(0)
  
neigh_df = pd.DataFrame.from_dict({"neighborhood_id": neigh_id,
                                  "name":neigh_name,
                                  "und_policial":neigh_police})


# load the GeoJSON data and use 'UTF-8'encoding
geo_json_natal = json.load(open('natal.geojson'))

neighborhood = []
# list all neighborhoods
for neigh in geo_json_natal['features']:
    neighborhood.append(neigh['properties']['name'])

# Prepare the data

# some neighborhoods names are different in IBGE API and GeoJSON file
neigh_df.loc[neigh_df.name == "Pitimbú",'name'] = "Pitimbu"
neigh_df.loc[neigh_df.name == "Mãe Luíza",'name'] = "Mãe Luiza"
neigh_df.loc[neigh_df.name == "Filipe Camarão",'name'] = "Felipe Camarão"
neigh_df.loc[neigh_df.name == "Guarapés",'name'] = "Guarapes"

neigh_df.set_index('name',inplace=True)
neigh_df.index.name = None
neigh_df["name"] = neigh_df.index
neigh_df.head()

# Create a colormap bar

from branca.colormap import linear

# colormap yellow and green (YlGn)
colormap = linear.YlGn_03.scale(
    neigh_df.und_policial.min(),
    neigh_df.und_policial.max())

colormap.caption="#Unidades Policiais in Natal-RN"

import numpy as np

# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=11,
    tiles='Stamen Terrain', width='85%',height='85%'
)

# create a threshold of legend
bins = np.linspace(neigh_df.und_policial.min(),
                   neigh_df.und_policial.max(),
                   6).tolist()


folium.Choropleth(
    geo_data=geo_json_natal,
    data=neigh_df,
    name= "Unidades Policiais",
    columns=['name', 'und_policial'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='#Unidades policiais in Natal-RN',
    bins=bins
).add_to(m)


folium.LayerControl().add_to(m)

m



